<span style="font-size: 20px;">**A/B-тестирование**</span>

Цель исследования: провести оценку результатов A/B-теста, который был проведен, чтобы проверить эффективность изменений на сайте или в приложении, и выяснить, какие изменения приводят к улучшению пользовательского опыта и повышению конверсии. Основная цель A/B-теста - проверить, будет ли новый вариант (группа B) лучше работать, чем старый вариант (группа A). В результате исследования мы должны определить, какой вариант (A или B) лучше работает, а также дать рекомендации по улучшению пользовательского опыта и повышению конверсии.

### Загрузка и предобработка данных 

In [184]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta, date
from scipy import stats as st
import numpy as np
import math as mth
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
from pandas.plotting import register_matplotlib_converters

In [185]:
try:
    ab_project_marketing_events = (
        pd.read_csv('C:/Users/Alexey Kuznetsov/Desktop/Новая папка/Untitled Folder/ab_project_marketing_events.csv'))
except:
    ab_project_marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
      
ab_project_marketing_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


Таблица "ab_project_marketing_events" — календарь маркетинговых событий на 2020 год

In [186]:
try:
    final_ab_new_users = (
        pd.read_csv('C:/Users/Alexey Kuznetsov/Desktop/Новая папка/Untitled Folder/final_ab_new_users.csv'))
except:
    final_ab_new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
      
final_ab_new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


Таблица "final_ab_new_users" отражает информацию о пользователях, зарегистрировавшихся с 7 до 21 декабря 2020 года.

In [187]:
try:
    final_ab_events = (
        pd.read_csv('C:/Users/Alexey Kuznetsov/Desktop/Новая папка/Untitled Folder/final_ab_events.csv'))
except:
    final_ab_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
      
final_ab_events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


Таблица "final_ab_events" содержит информацию о действиях новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

In [188]:
try:
    final_ab_participants = (
        pd.read_csv('C:/Users/Alexey Kuznetsov/Desktop/Новая папка/Untitled Folder/final_ab_participants.csv'))
except:
    final_ab_participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')
      
final_ab_participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


"final_ab_participants" -  таблица участников тестов

In [189]:
ab_project_marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [190]:
# изменение типа данных колонок 'start_dt' и 'finish_dt' на datetime

ab_project_marketing_events['start_dt'] = (
    pd.to_datetime(ab_project_marketing_events['start_dt']))

ab_project_marketing_events['finish_dt'] = (
    pd.to_datetime(ab_project_marketing_events['finish_dt']))


In [191]:
final_ab_new_users.duplicated().sum()

0

In [192]:
final_ab_new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [193]:
# изменение типа данных колонок 'first_date' на datetime

final_ab_new_users['first_date'] = (
    pd.to_datetime(final_ab_new_users['first_date']))

В таблице final_ab_new_users дубликаты и пропуски отсутствуют

In [194]:
final_ab_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [195]:
final_ab_events['event_dt'] = (
    pd.to_datetime(final_ab_events['event_dt']))

final_ab_events.duplicated().sum()

0

В таблице final_ab_events пропуски в столбце "details", в котором содержатся дополнительные данные о событии. Следует оставить как есть, т.к. заполнение этого столбца необязательно.

In [196]:
final_ab_participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [197]:
final_ab_participants.duplicated().sum()

0

В таблице final_ab_participants дубликаты и пропуски отсутствуют.

В процессе предобработки данных были изучены 4 датасета - final_ab_participants, final_ab_events, final_ab_new_users, ab_project_marketing_events. Дубликаты и пропуски отстутствуют. Изменены только типы данных колонок с датами.

###  Оценка корректности проведения теста

#### Техническое задание

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: в тест должно быть отобрано 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.    

In [198]:
final_ab_participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64

In [199]:
#пользователи, учавствующие в двух тестах

user_test = final_ab_participants.groupby('user_id')['ab_test'].nunique()
user_test = user_test[user_test > 1]

final_ab_participants.query('user_id in @user_test.index').query('ab_test == "recommender_system_test"')

,user_id,group,ab_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
10,B3A2485649E4A012,A,recommender_system_test
25,EAFB9027A27D510C,B,recommender_system_test
29,5D5E6EE92AF6E9E0,B,recommender_system_test
...,...,...,...
6323,C2DC7B6881CE6E30,B,recommender_system_test
6338,EA29547AB3C0CB9C,B,recommender_system_test
6341,9A44E27079666291,B,recommender_system_test
6343,FA86D8DFAD3ADADE,A,recommender_system_test


In [200]:
#пользователи, учавствующие в двух группах теста "recommender_system_test"

group_user = final_ab_participants.query('ab_test == "recommender_system_test"').groupby('user_id')['group'].nunique()

group_user[group_user > 1]

Series([], Name: group, dtype: int64)

In [201]:
group_count = (final_ab_participants
 .query('ab_test == "recommender_system_test"')
 .groupby('group').agg({'user_id' : 'count'})
 .assign(share=lambda x: x / x.sum() *100)
 .reset_index())

colors = ['LightSkyBlue', 'lightsalmon']
labels = ['A', 'B']
fig = go.Figure(data=[go.Pie(labels=labels, values=group_count['share'])])
fig.update_layout(title_text='Соотношение пользователей в группах')
fig.update_traces(marker=dict(colors=colors))
fig.show()

In [202]:
#оставить пользователей только необходимого теста

final_ab_participants = final_ab_participants[final_ab_participants.ab_test == "recommender_system_test"]

In [203]:
print('Дата запуска - ', final_ab_events['event_dt'].min())

Дата запуска -  2020-12-07 00:00:33


In [204]:
print('Дата остановки набора новых пользователей -', final_ab_new_users['first_date'].max())

Дата остановки набора новых пользователей - 2020-12-23 00:00:00


In [205]:
new_users_participants = final_ab_participants.merge(final_ab_new_users, on='user_id', how='left')

new_users_participants.query('first_date > "2020-12-21"')

,user_id,group,ab_test,first_date,region,device


Пользователи с датой регистрации позже 2020-12-21 отсутствуют

Согласно ТЗ дата остановки набора - 2020-12-21. 

In [206]:
print('Дата остановки - ', final_ab_events['event_dt'].max())

Дата остановки -  2020-12-30 23:36:33


In [207]:
#убрать пользователей, зарегистрировавшихся после 2020-12-21

final_ab_new_users = final_ab_new_users[final_ab_new_users.first_date <= "2020-12-21"]

In [208]:
final_ab_new_users.groupby('region').agg({'user_id' : 'count'})

,user_id
region,
APAC,2883
CIS,2900
EU,42340
N.America,8347


In [209]:
#проверить процент пользователей из EU

region_eu = final_ab_new_users.query('region == "EU"')['user_id']

final_ab_participants = final_ab_participants.query('user_id in @region_eu')

print('Процент пользователей из EU - ',((final_ab_participants['user_id'].count()) /
 (final_ab_new_users.query('region == "EU"')['user_id'].count()) *100))

Процент пользователей из EU -  15.0


In [210]:
#удалить всех пользователей, которые попали в оба теста, но в тесте "recommender_system_test" находящихся в группе B

final_ab_participants = final_ab_participants.query('user_id not in @user_test.index or group != "B"')

In [211]:
print('Количество участников теста - ', final_ab_participants['user_id'].nunique())

Количество участников теста -  5670


In [212]:
#объединить таблицы "final_ab_participants" и "final_ab_new_users"

final_ab_participants = final_ab_participants.merge(final_ab_new_users, on='user_id', how='left')

In [213]:
#объединить таблицы "final_ab_participants" и "final_ab_events"

final_ab_participants = final_ab_participants.merge(final_ab_events, on='user_id', how='left')

In [214]:
# удалить события, произошедшие позже 14 дней после регистрации

final_ab = (
    final_ab_participants.loc[final_ab_participants['event_dt']<= (final_ab_participants['first_date']+ timedelta(days=14))])

In [215]:
final_ab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21379 entries, 0 to 24503
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     21379 non-null  object        
 1   group       21379 non-null  object        
 2   ab_test     21379 non-null  object        
 3   first_date  21379 non-null  datetime64[ns]
 4   region      21379 non-null  object        
 5   device      21379 non-null  object        
 6   event_dt    21379 non-null  datetime64[ns]
 7   event_name  21379 non-null  object        
 8   details     2946 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.6+ MB


In [216]:
# маркетинговые активности во время теста

(ab_project_marketing_events.loc[ab_project_marketing_events['start_dt'] >= final_ab['event_dt'].min()])

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
10,CIS New Year Gift Lottery,CIS,2020-12-30,2021-01-07


#### Выводы.

Выявлены пересечения с конкурирующим тестом: всего 1602 пользователя. Решено удалить только тех пользователей, которые находятся в группе B необходимого теста. 

Пользователи, учавствующие в двух группах теста одновременно, не найдены. 

Во время проведения теста проводилась одна маркетинговая акция "Christmas&New Year Promo".

Данные приведены в соответствие с ТЗ. Однако, после удаления событий, произошедших позже 14 дней после регистрации, а также пользователей, зарегистрировавшихся после 21 декабря, количество участников сильно уменьшилось и больше не удовлетворяет соответствующему пункту ТЗ (о количестве участников теста).  

In [217]:
final_ab['lifetime'] = final_ab['event_dt'] - final_ab['first_date']

final_ab.loc[final_ab['lifetime'] > timedelta(days=14)]

C:\Users\Alexey Kuznetsov\AppData\Local\Temp\ipykernel_7888\1666442483.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,lifetime


### Исследовательский анализ данных

In [218]:
events = final_ab.groupby(['group', 'user_id']).agg({'event_name' : 'count'}).reset_index()

In [219]:
# средние и медианные значения количества событий в разных группах

events.groupby('group').agg({'event_name' : ['count', 'mean', 'median']})


event_name                 
           count      mean median
group                            
A           2604  6.849078    6.0
B            655  5.410687    4.0

In [220]:
final_ab['event_dt'] = final_ab['event_dt'].dt.date
events_dt = final_ab.groupby(['group', 'event_dt']).agg({'event_name' : 'count'}).reset_index()


fig = go.Figure(data=[
    go.Bar(name='A', 
           x=events_dt.loc[events_dt['group'] == "A", 'event_dt'],
           y=events_dt.loc[events_dt['group'] == "A", 'event_name'],
           text=events_dt.loc[events_dt['group'] == "A", 'event_name'],
           textposition='auto',
           marker_color='LightSkyBlue'),
    go.Bar(name='B', 
           x=events_dt.loc[events_dt['group'] == "B", 'event_dt'], 
           y=events_dt.loc[events_dt['group'] == "B", 'event_name'],
           text=events_dt.loc[events_dt['group'] == "B", 'event_name'],
           textposition='auto',
           marker_color='lightsalmon')
])

fig.update_layout(height=800, width=1000, barmode='group', title_text= 'Распределение событий по дням')
fig.update_yaxes(title_text='события')
fig.show()

C:\Users\Alexey Kuznetsov\AppData\Local\Temp\ipykernel_7888\3000216182.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [221]:
users_count =final_ab.groupby(['group', 'event_name']).agg({'user_id': 'nunique'}).reset_index()

users_count = (
    pd.concat([users_count[users_count['event_name']!="purchase"]
               .sort_values(by='user_id',ascending=False), users_count[users_count['event_name']=="purchase"]]))
    
fig = go.Figure()
fig.add_trace(go.Funnel(
    name = 'A',
    y = users_count['event_name'].unique(),
    x = users_count.loc[users_count['group'] == "A", 'user_id'] ,
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'B',
    orientation = "h",
    y = users_count['event_name'].unique(),
    x = users_count.loc[users_count['group'] == "B", 'user_id'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.update_layout(title='Воронка событий')
fig.show()  

#### Выводы

Всего в группе А - 2604 пользователя, среднее количество событий на пользователя - 6,8, медианное значение событий - 6.
Всего в группе А - 655 пользователя, среднее количество событий на пользователя - 5,4, медианное значение событий - 4.

События по дням распределены неравномерно. 21 декабря - сильный рост событий во всех группах, далее идет снижение к концу года - это связанно с праздниками, как правило, происходит каждый год. 

Конверсия в событие "purchase" в группе А  составила 32%, а в группе В - 29%

### Результаты A/B-тестирования

Необходимо выяснить будет ли отличие между группами статистически достоверным (в разрезе событий)

Гипотезы:

H_0: Между долями нет значимой разницы

H_1: Между долями есть значимая разница

Решено откорректировать уровень статистической значимости, используя метод Бонферрони, т.к. он позволяет уменьшить вероятность совершения ошибки первого рода (отвержение верной нулевой гипотезы) в множественных сравнениях.

0,05/3 = 0,0167

In [222]:
def z_test(a, b, c, d, alpha = 0.0167):
    p1 = a/c
    p2 = b/d
    
    print(a, b, c , d)
    p_combined = (a + b) / (c + d)
    difference = p1 - p2

    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/c + 1/d))
    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if  p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными'
        )

In [223]:
events = ['product_page', 'product_cart', 'purchase']

def test(a, b):    
    for event in events:

        group_count1 = final_ab.loc[final_ab['group'] == a, 'user_id'].nunique()
        group_count2 = final_ab.loc[final_ab['group'] == b, 'user_id'].nunique()  
        event_count1 = final_ab.loc[(final_ab['group'] == a) 
                                            & (final_ab['event_name'] == event), 'user_id'].nunique()
        event_count2 = final_ab.loc[(final_ab['group'] == b)
                                            & (final_ab['event_name'] == event), 'user_id'].nunique()
        
        table=pd.DataFrame(columns=['group', 'event_count', 'group_count'])
        table['group'] = ['A', 'B']
        table['event_count'] = [event_count1, event_count2]
        table['group_count'] = [group_count1, group_count2]
        
        display(event, table)
        
        z_test(event_count1, event_count2, group_count1, group_count2)
        

In [224]:
test('A','B')

'product_page'

,group,event_count,group_count
0,A,1685,2604
1,B,367,655


1685 367 2604 655
p-значение:  3.940541072622317e-05
Отвергаем нулевую гипотезу: между долями есть значимая разница


'product_cart'

,group,event_count,group_count
0,A,782,2604
1,B,184,655


782 184 2604 655
p-значение:  0.3313501610605525
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


'purchase'

,group,event_count,group_count
0,A,833,2604
1,B,191,655


833 191 2604 655
p-значение:  0.16326348032632865
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


#### Выводы

В результате А/В-тестирования была проверена статистическая значимость разницы долей с помощью z-теста. 

Данная проверка позволяет оценить, является ли различие между группами случайным или значимым. Если различия между группами являются статистически значимыми, то это может указывать на наличие реального эффекта в тестируемом параметре. Таким образом проверка необходима определения эффективности тестируемых воздействий и принятия решений на основе этих данных.

Всего проведено 3 проверки, из которых только одна показала различие в событии "product page". Так как согласно воронке событий только 28% доходят до события 'product_cart' и 29% - до 'purchase', можно считать что внедрение улучшенной рекомендательной системы пока не привело к желаемым изменениям в конверсии. 